In [1]:
from parse_deck import parse_events_from_official

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

import time
import re
from tqdm import tqdm

import pandas as pd
from collections import OrderedDict

import json
import os

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--start-maximized")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
def wait_loading_circle(timeout: int = 20):
    WebDriverWait(driver, 20).until(EC.invisibility_of_element_located((By.XPATH, "//div[@class='sk-circle-container']")))
    

In [4]:
# decks = {
#     deck category 1: [
#         {
#             deck_link: "",
#             deck_code: "",
#             pokemons: {},
#             tools: {},
#             supporters: {},
#             stages: {},
#             energies: {},
#             rank: 1,
#             num_people: 32,
#             date: datetime
#         }, ...
#     ],
#     deck category 2:...
# }
all_categories = {
    "simple": ["ルギアVSTAR", "ミュウVMAX"],  # if this card exists, then we could decide the category
    "complicate": ["ゾロア", "レジドラゴVSTAR", "ギラティナVSTAR"]
}

decks = {}
store_file_name = 'store.json'
if os.path.exists(store_file_name):
    with open(store_file_name, 'r') as f:
        decks = json.load(f)

store_code_list = []
for category in decks.keys():
    for d in decks[category]:
        store_code_list.append(d["deck_code"])

In [5]:
len(store_code_list)

0

In [6]:
# parse events and save to json
parse_events_from_official(decks, all_categories, store_code_list)

with open(store_file_name, 'w') as f:
    json.dump(decks, f, ensure_ascii=False, indent=4)

 20%|████████▊                                   | 4/20 [00:35<02:34,  9.65s/it]

Message: no such element: Unable to locate element: {"method":"tag name","selector":"a"}
  (Session info: headless chrome=108.0.5359.94)
Stacktrace:
0   chromedriver                        0x000000010e1a2f38 chromedriver + 4910904
1   chromedriver                        0x000000010e122a03 chromedriver + 4385283
2   chromedriver                        0x000000010dd67747 chromedriver + 472903
3   chromedriver                        0x000000010ddac34c chromedriver + 754508
4   chromedriver                        0x000000010ddac5a1 chromedriver + 755105
5   chromedriver                        0x000000010dd9e216 chromedriver + 696854
6   chromedriver                        0x000000010ddd213d chromedriver + 909629
7   chromedriver                        0x000000010dd9e0ed chromedriver + 696557
8   chromedriver                        0x000000010ddd22ce chromedriver + 910030
9   chromedriver                        0x000000010dded28e chromedriver + 1020558
10  chromedriver                      

100%|███████████████████████████████████████████| 20/20 [02:45<00:00,  8.26s/it]


In [7]:
# modified categories
decks.keys()

dict_keys(['others', 'ルギアVSTAR', 'ミュウVMAX', 'LOST ギラティナVSTAR', 'レジドラゴVSTAR', 'アルレジドラゴVSTAR', 'ゾロア', 'Other ギラティナVSTAR'])

In [32]:
target_category = "ゾロア"
df_list = []
common_cols = ["date", "num_people", "rank"]

for card_type in ["pokemons", "tools", "supporters", "stages", "energies"]:
    # df init
    df = pd.DataFrame()
    for _, deck in enumerate(decks[target_category]):
        deck_code = deck["deck_code"]  # row id
        pokecard = OrderedDict()
        pokecard["date"] = deck["date"]
        pokecard["num_people"] = deck["num_people"]
        pokecard["rank"] = deck["rank"]
        pokecard.update(deck[card_type])
        if _ == 0:
            df = pd.DataFrame(pokecard, index=[deck_code])
        else:
            df = pd.concat([df, pd.DataFrame(pokecard, index=[deck_code])])
    df = df.fillna(0)
    
    # sort rows by date
    df = df.sort_values(by=['date'], ascending=False)

    # select cols for analysis
    col_list = list(df)
    for c in common_cols: col_list.remove(c)

    # calculate
    num_decks = df.shape[0]
    num_used = df[col_list].sum(axis='rows', numeric_only=True)
    num_picked = df[col_list].astype(bool).sum(axis='rows')
    avg_num_used = num_used / num_picked
    pick_rate = num_picked / num_decks
    
    # insert rows in df
    df.loc["avg_num_used"] = {}
    df.loc["pick_rate"] = {}
    for col in col_list:
        df.loc["avg_num_used", col] = avg_num_used[col]
        df.loc["pick_rate", col] = pick_rate[col]

    # reorder index in df, move 'avg_num_used' and 'pick_rate' to top
    num_rows = df.shape[0]
    target_rows = [num_rows-1, num_rows-2]
    idx = target_rows + [i for i in range(len(df)) if i not in target_rows]
    df = df.iloc[idx]
    
    # sort cols by pick rate
    df = df.sort_values('pick_rate', axis=1, ascending=False)
    col_list = list(df)
    for c in common_cols: col_list.remove(c)
    df = df[common_cols + col_list]
    
    # store
    df_list.append(df)

In [33]:
len(decks[target_category])

11

In [34]:
df_list[0]

,date,num_people,rank,ゾロアーク,ゾロア,チラーミィ,チラチーノ,タルップル,ライチュウ,テールナー,グラエナ,ヤドラン,マナフィ,アップリュー,バサギリ,かがやくジラーチ,ヒスイ ゾロアーク,ミノマダム,ピカチュウ
pick_rate,NaN,NaN,NaN,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,0.818182,0.818182,0.636364,0.181818,0.090909,0.090909
avg_num_used,NaN,NaN,NaN,4.0,4.0,3.363636,2.909091,1.0,1.090909,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
kfkVfF-YHcBw1-FFfkkk,2022年12月04日(日),60.0,5.0,4.0,4.0,4.000000,3.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
Mppyyy-gUfbO4-pyyM2p,2022年12月04日(日),32.0,2.0,4.0,4.0,3.000000,3.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
DDcDc8-1Sm4vR-88888x,2022年12月03日(土),48.0,3.0,4.0,4.0,3.000000,3.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FkVbFw-SNJ1nv-kVFfkF,2022年12月03日(土),64.0,1.0,4.0,4.0,4.000000,3.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
DcaJ8x-YkIgSP-488c8x,2022年11月27日(日),32.0,5.0,4.0,4.0,3.000000,3.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
D8KDG8-YjU3tt-888xxx,2022年11月27日(日),32.0,5.0,4.0,4.0,3.000000,2.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
kkVkVk-0R1HoS-kFvv1V,2022年11月26日(土),32.0,5.0,4.0,4.0,4.000000,3.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
yyMyMy-B9ID3t-yMXSyM,2022年11月26日(土),32.0,1.0,4.0,4.0,3.000000,3.000000,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000


In [35]:
df_list[1]

,date,num_people,rank,レベルボール,しんかのおこう,ふつうのつりざお,レスキューキャリー,クイックボール,こだわりベルト,ハイパーボール,やまびこホーン,ヒスイのヘビーボール,ロストスイーパー,バトルVIPパス,あなぬけのヒモ,ともだちてちょう,ふうせん
pick_rate,NaN,NaN,NaN,1.0,1.000000,1.0,0.818182,0.545455,0.545455,0.545455,0.545455,0.272727,0.272727,0.181818,0.090909,0.090909,0.090909
avg_num_used,NaN,NaN,NaN,4.0,3.454545,2.0,1.555556,2.166667,1.000000,2.666667,1.000000,1.000000,1.000000,4.000000,1.000000,1.000000,1.000000
kfkVfF-YHcBw1-FFfkkk,2022年12月04日(日),60.0,5.0,4.0,4.000000,3.0,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Mppyyy-gUfbO4-pyyM2p,2022年12月04日(日),32.0,2.0,4.0,3.000000,2.0,2.000000,3.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
DDcDc8-1Sm4vR-88888x,2022年12月03日(土),48.0,3.0,4.0,3.000000,2.0,1.000000,3.000000,1.000000,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FkVbFw-SNJ1nv-kVFfkF,2022年12月03日(土),64.0,1.0,4.0,4.000000,3.0,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
DcaJ8x-YkIgSP-488c8x,2022年11月27日(日),32.0,5.0,4.0,4.000000,2.0,1.000000,2.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
D8KDG8-YjU3tt-888xxx,2022年11月27日(日),32.0,5.0,4.0,4.000000,2.0,2.000000,0.000000,0.000000,4.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
kkVkVk-0R1HoS-kFvv1V,2022年11月26日(土),32.0,5.0,4.0,4.000000,2.0,2.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
yyMyMy-B9ID3t-yMXSyM,2022年11月26日(土),32.0,1.0,4.0,3.000000,1.0,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,1.000000,1.000000,0.000000


In [36]:
df_list[2]

,date,num_people,rank,博士の研究,セレナ,ボスの指令,マリィ,クララ,さぎょういん,シロナの覇気,シバ,ネズ,カトレア
pick_rate,NaN,NaN,NaN,1.000000,1.0,0.818182,0.727273,0.181818,0.090909,0.090909,0.090909,0.090909,0.090909
avg_num_used,NaN,NaN,NaN,2.909091,4.0,1.111111,2.250000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
kfkVfF-YHcBw1-FFfkkk,2022年12月04日(日),60.0,5.0,3.000000,4.0,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Mppyyy-gUfbO4-pyyM2p,2022年12月04日(日),32.0,2.0,4.000000,4.0,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
DDcDc8-1Sm4vR-88888x,2022年12月03日(土),48.0,3.0,3.000000,4.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FkVbFw-SNJ1nv-kVFfkF,2022年12月03日(土),64.0,1.0,3.000000,4.0,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
DcaJ8x-YkIgSP-488c8x,2022年11月27日(日),32.0,5.0,2.000000,4.0,0.000000,3.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
D8KDG8-YjU3tt-888xxx,2022年11月27日(日),32.0,5.0,3.000000,4.0,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
kkVkVk-0R1HoS-kFvv1V,2022年11月26日(土),32.0,5.0,3.000000,4.0,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
yyMyMy-B9ID3t-yMXSyM,2022年11月26日(土),32.0,1.0,3.000000,4.0,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [37]:
df_list[3]

,date,num_people,rank,頂への雪道
pick_rate,NaN,NaN,NaN,0.909091
avg_num_used,NaN,NaN,NaN,1.800000
kfkVfF-YHcBw1-FFfkkk,2022年12月04日(日),60.0,5.0,2.000000
Mppyyy-gUfbO4-pyyM2p,2022年12月04日(日),32.0,2.0,2.000000
DDcDc8-1Sm4vR-88888x,2022年12月03日(土),48.0,3.0,2.000000
FkVbFw-SNJ1nv-kVFfkF,2022年12月03日(土),64.0,1.0,2.000000
DcaJ8x-YkIgSP-488c8x,2022年11月27日(日),32.0,5.0,2.000000
D8KDG8-YjU3tt-888xxx,2022年11月27日(日),32.0,5.0,1.000000
kkVkVk-0R1HoS-kFvv1V,2022年11月26日(土),32.0,5.0,2.000000
yyMyMy-B9ID3t-yMXSyM,2022年11月26日(土),32.0,1.0,2.000000


In [38]:
df_list[4]

,date,num_people,rank,ツインエネルギー,ダブルターボエネルギー,キャプチャーエネルギー,基本超エネルギー,ギフトエネルギー
pick_rate,NaN,NaN,NaN,1.0,1.0,1.000000,0.090909,0.090909
avg_num_used,NaN,NaN,NaN,4.0,2.0,2.909091,2.000000,1.000000
kfkVfF-YHcBw1-FFfkkk,2022年12月04日(日),60.0,5.0,4.0,3.0,3.000000,0.000000,0.000000
Mppyyy-gUfbO4-pyyM2p,2022年12月04日(日),32.0,2.0,4.0,3.0,2.000000,0.000000,0.000000
DDcDc8-1Sm4vR-88888x,2022年12月03日(土),48.0,3.0,4.0,2.0,3.000000,0.000000,0.000000
FkVbFw-SNJ1nv-kVFfkF,2022年12月03日(土),64.0,1.0,4.0,3.0,3.000000,0.000000,0.000000
DcaJ8x-YkIgSP-488c8x,2022年11月27日(日),32.0,5.0,4.0,1.0,3.000000,0.000000,0.000000
D8KDG8-YjU3tt-888xxx,2022年11月27日(日),32.0,5.0,4.0,2.0,4.000000,0.000000,0.000000
kkVkVk-0R1HoS-kFvv1V,2022年11月26日(土),32.0,5.0,4.0,1.0,4.000000,0.000000,0.000000
yyMyMy-B9ID3t-yMXSyM,2022年11月26日(土),32.0,1.0,4.0,2.0,2.000000,0.000000,0.000000


In [39]:
len(decks["others"])

272